# **Receipt Extraction MCP Server using Amazon Bedrock AgentCore Runtime, Langchain and Gemini 2.5 Flash**

This MCP server help you extract receipt photo information such as store name, purchase date and total/amount then write output of extract information to Amazon DynamoDB table and send email of output of extract information using Amazon SNS. Then test this MCP server with Langchain.

In [1]:
from google.colab import userdata
import os
# Retrieve AWS credentials from Colab Secrets
os.environ["AWS_ACCESS_KEY_ID"] = userdata.get('AWSACCESSKEY')
os.environ["AWS_SECRET_ACCESS_KEY"] = userdata.get('AWSSECRETKEY')

# **Creating Receipt Extraction MCP Server**

In [2]:
%%writefile mcp_server.py
from fastmcp import FastMCP
from PIL import Image
from google import genai
from pydantic import BaseModel, Field
from google.genai import types
from decimal import Decimal
import boto3
import json
import os
import tempfile

# MCP Server using FastMCP
mcp = FastMCP("Receipt Extraction MCP Server", stateless_http=True, host="0.0.0.0")

region = "us-west-2"
s3 = boto3.client('s3', region_name=region)

# Retrieve Gemini API Key from AWS Secret Manager
secretmanager = boto3.client('secretsmanager', region_name=region)
response = secretmanager.get_secret_value(SecretId='geminiapikey')
secret_json = json.loads(response["SecretString"])
apikey = secret_json["GEMINI_API_KEY"]
geminiclient = genai.Client(api_key=apikey)

# Retrieve AWS Account ID for Amazon SNS TopicARN
sts_client = boto3.client('sts')
response = sts_client.get_caller_identity()
aws_account_id = response['Account']

@mcp.tool()
def receiptExtraction(inputphoto: str):
    class ReceiptExtractionResult(BaseModel):
        """Extracted receipt information."""
        storeName: str = Field(description="Name of store or store name. Must uppercase.")
        purchaseDate: str = Field(description="Purchase date with \"DD-MM-YYYY\" format date.")
        total: int | float = Field(description="Total or amount. Number and (.) only without any such as $ or other currency.")

    file_name, file_extension = os.path.splitext(inputphoto)
    with tempfile.NamedTemporaryFile(delete=False, suffix=file_extension) as temp_photo_file:
        local_file_path = temp_photo_file.name
        s3.download_file("receipts-extraction", inputphoto, local_file_path)

    image = Image.open(local_file_path)
    prompts = """
    Extract store name, purchase date and total/amount based this receipt photo with structure output based ReceiptExtractionResult class format.
    """
    gemini_response = geminiclient.models.generate_content(
        model="gemini-2.5-flash",
        contents=[image, prompts],
        config={
            "response_mime_type": "application/json",
            "response_json_schema": ReceiptExtractionResult.model_json_schema(),
        },
    )
    response_parsed = ReceiptExtractionResult.model_validate_json(gemini_response.text)
    os.remove(local_file_path)
    s3.delete_object(Bucket="receipts-extraction", Key=inputphoto)
    return {"storeName": response_parsed.storeName, "date": response_parsed.purchaseDate, "total": response_parsed.total}

@mcp.tool()
def writeOutput(storeName: str, purchase_date: str, total: float | int):
    """Write receipt extraction results to Amazon DynamoDB"""
    dynamodb = boto3.resource('dynamodb', region_name="us-west-2")
    table_name = "receiptsExtraction"
    table = dynamodb.Table(table_name)
    table.put_item(Item={
        'storeName': storeName,
        'date': purchase_date,
        'total': Decimal(str(total)) if isinstance(total, float) else total
    })
    return {"message": f"Check your {table_name} table in Amazon DynamoDB"}

@mcp.tool()
def sendEmail(storeName: str, purchase_date: str, total: float | int):
    """Send email notification with receipt extraction results using Amazon SNS"""
    sns = boto3.client('sns', region_name="us-west-2")
    sns.publish(
        TopicArn=f"arn:aws:sns:us-west-2:{aws_account_id}:receiptsExtractionEmail",
        Message=f"Output of this receipt photo:\n\nStore Name: {storeName}\n\nPurchase Date: {purchase_date}\n\nGrand Total: {total}",
        Subject=f"Result of Receipt Extraction - {storeName}"
    )
    return {"message": f"Check your email with subject Result of Receipt Extraction - {storeName}"}

mcp.run(transport="streamable-http")

Writing mcp_server.py


In [3]:
%%writefile requirements.txt
mcp
fastmcp
bedrock-agentcore
boto3
google-genai
langchain[google-genai]
pydantic
Pillow

Writing requirements.txt


# **Create Amazon Cognito User Pool for Inbound Authentication**

In [ ]:
# Install all libraries
!pip install -r requirements.txt bedrock-agentcore-starter-toolkit langchain-mcp-adapters

In [4]:
import boto3
region = "us-west-2"

# Initialize Cognito client
cognito = boto3.client('cognito-idp', region)

# Create Cognito User Pool
user_pool_response = cognito.create_user_pool(
    PoolName='Receipt Extraction MCP Server Pool',
    Policies={
        'PasswordPolicy': {
            'MinimumLength': 7
        }
    }
)
cognito_pool_id = user_pool_response['UserPool']['Id']

# Create Cognito User Pool Client
app_client_response = cognito.create_user_pool_client(
    UserPoolId=cognito_pool_id,
    ClientName='Receipt Extraction MCP Server Pool Client',
    GenerateSecret=False,
    ExplicitAuthFlows=[
        'ALLOW_USER_PASSWORD_AUTH',
        'ALLOW_REFRESH_TOKEN_AUTH'
    ]
)
cognito_client_id = app_client_response['UserPoolClient']['ClientId']

# **Amazon Bedrock AgentCore Runtime Configuration with MCP Protocol and Auth Configuration from Amazon Cognito User Pool**

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
agentcore_runtime = Runtime()
agent_name="gemini_mcp_server"

response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
    protocol="MCP",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [cognito_client_id],
            "discoveryUrl": f"https://cognito-idp.{region}.amazonaws.com/{cognito_pool_id}/.well-known/openid-configuration",
        }
    }
)
response

In [6]:
print("AgentCore Runtime for MCP is configured.")

AgentCore Runtime for MCP is configured.


# **Deploy Receipt Extraction MCP Server to Amazon Bedrock AgentCore Runtime**

In [ ]:
launch_result = agentcore_runtime.launch()

In [8]:
print("AgentCore Runtime for MCP is now available.")

AgentCore Runtime for MCP is now available.


# **Temporary Stop! Your AgentCore Runtime now available but don't invoke directly because your Gemini API Key in AWS Secret Manager, Amazon DynamoDB table and Amazon SNS subscription not yet added to your IAM role for AgentCore Runtime**

**1. Go to AWS Secret Manager, click secret name then copy Secret ARN of Gemini API Key.**

**2. Go to Amazon Bedrock AgentCore -> Agent runtime then click your agent name that already created. Click "Version 1" then click IAM service role of Permissions that automatically opened a new tab (e.g. AmazonBedrockAgentCoreSDKRuntime-{region-name}-{random-number-letter}).**

**3. Click IAM policy name that related (e.g. BedrockAgentCoreRuntimeExecutionPolicy-{your-agent-name}).**

**4. Add your Secret ARN of Gemini API Key in resource of "secretsManager:GetSecretValue" action, then add new statement, add new action and add resource for Amazon S3, Amazon DynamoDB and Amazon SNS then click Next then click Save. DONE**

# **Continue to invoke your AgentCore Runtime now**

# **Get Bearer Token from Amazon Cognito Authentication**

In [9]:
import random
import string

def generate_random_password(length=12):
    characters = string.ascii_letters + string.digits + string.punctuation
    password = ''.join(random.choice(characters) for i in range(length))
    return password
random_password_default = generate_random_password()

In [ ]:
# Create Cognito Admin User
cognito.admin_create_user(
    UserPoolId=cognito_pool_id,
    Username='testuser',
    TemporaryPassword='Temp123!',
    MessageAction='SUPPRESS'
)

# Create Cognito Admin Password
cognito.admin_set_user_password(
    UserPoolId=cognito_pool_id,
    Username='testuser',
    Password=random_password_default,
    Permanent=True
)

In [11]:
auth_response = cognito.initiate_auth(
    ClientId=cognito_client_id,
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={
        'USERNAME': 'testuser',
        'PASSWORD': random_password_default
    }
)
bearer_token = auth_response['AuthenticationResult']['AccessToken']

# **Invoke MCP Server using Langchain MCP Client with several sample photo file**

In [12]:
from google import genai
from langchain.chat_models import init_chat_model
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.messages import ToolMessage
from langchain.agents import create_agent
import boto3
import json
import logging

region = "us-west-2"
s3 = boto3.client('s3', region)

# Retrieve Gemini API Key from AWS Secret Manager
secretmanager = boto3.client('secretsmanager', region)
response = secretmanager.get_secret_value(SecretId='geminiapikey')
secret_json = json.loads(response["SecretString"])
api_key = secret_json["GEMINI_API_KEY"]
llm = init_chat_model("google_genai:gemini-2.5-flash", google_api_key=api_key)

def upload_to_s3(file_path):
    s3.upload_file(file_path, "receipts-extraction", file_path)
    return file_path

In [13]:
async def invokemcp(photo):
    # Suppress warning message
    logging.getLogger("mcp.client.streamable_http").setLevel(logging.ERROR)

    # Get AgentCore Runtime MCP Server ARN
    mcp_runtime_arn = launch_result.agent_arn
    encoded_arn = mcp_runtime_arn.replace(':', '%3A').replace('/', '%2F')

    print("Connecting to Receipt Extraction MCP Server...")

    # Create MCP client configuration
    client = MultiServerMCPClient(
        {
            "receipt_server": {
                "url": f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT",
                "headers": {
                    "Authorization": f"Bearer {bearer_token}",
                    "Content-Type": "application/json"
                },
                "transport": "http"
            }
        }
    )

    receipt = upload_to_s3(photo)

    # Get MCP server tool
    tools = await client.get_tools()
    print(f"\nAvailable tools: {[tool.name for tool in tools]}")

    # Create Langchain Agent with MCP server tool
    agent = create_agent(llm, tools)

    # Invoke agent with proper context
    print(f"\nProcessing receipt photo file: {receipt}")
    response = await agent.ainvoke({
        "messages": [
            f"Please extract the receipt information from {receipt}, write the output to DynamoDB, and send an email notification."
        ]
    })
    return response

In [15]:
# Invoke MCP server with sample photo file
result = await invokemcp("photos/oakstreet.jpg")
print(result)

Connecting to Receipt Extraction MCP Server...

Available tools: ['receiptExtraction', 'writeOutput', 'sendEmail']

Processing receipt photo file: photos/oakstreet.jpg
{'messages': [HumanMessage(content='Please extract the receipt information from photos/oakstreet.jpg, write the output to DynamoDB, and send an email notification.', additional_kwargs={}, response_metadata={}, id='f6e3476f-7158-4bb2-84ff-cb5e964d5073'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'receiptExtraction', 'arguments': '{"inputphoto": "photos/oakstreet.jpg"}'}, '__gemini_function_call_thought_signatures__': {'7ab1e054-51fb-4c4a-bda5-829b43cf2504': 'CvcCAXLI2nzlWAIGYFpsq7cNlBa6Qkt+aioB4ufKL5nLkEq8Ku5oZYGPjNs2oN6ImcOmFlJoACoXWwFUCE4+xRjLa+UnaV+kZyV0p1uOaeyfIzXT2lIdgO5DamJlyHyZ4JGg74JkFfQpZKWc88NKJ6aFHpGyoy8A4boPVOAY8WP7EwboE8fFzhHgHHfxmqxFIvrhkXRuM+w+3x4YGvCtiqwkQJWXHzH1MR2Y/VlmZ7gHkP4kpL6B3jG69QAGAX5BhwTq5SnS/6LtYWdUOEfelWhbvOJ8ic4qghLmUlKklotvgHC6Q84YwlhouSjcKq3E+pXMY0xPW2jQo1+5T0p84mCZi

In [16]:
# Extract structured content from tool messages
for message in result["messages"]:
    if isinstance(message, ToolMessage) and message.artifact:
        structured_content = message.artifact["structured_content"]
        print(structured_content)

{'storeName': 'OAK STREET MARKET', 'date': '26-10-2023', 'total': 42.58}
{'message': 'Check your receiptsExtraction table in Amazon DynamoDB'}
{'message': 'Check your email with subject Result of Receipt Extraction - OAK STREET MARKET'}


In [17]:
# Invoke MCP server with sample photo file
result = await invokemcp("photos/ovenfc.jpg")
print(result)

Connecting to Receipt Extraction MCP Server...

Available tools: ['receiptExtraction', 'writeOutput', 'sendEmail']

Processing receipt photo file: photos/ovenfc.jpg
{'messages': [HumanMessage(content='Please extract the receipt information from photos/ovenfc.jpg, write the output to DynamoDB, and send an email notification.', additional_kwargs={}, response_metadata={}, id='ffeee257-ef23-4c87-89bc-01f8c4dbec94'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'receiptExtraction', 'arguments': '{"inputphoto": "photos/ovenfc.jpg"}'}, '__gemini_function_call_thought_signatures__': {'73a8946f-8cd3-4561-85df-af6dd7adaf35': 'CvMCAXLI2nwCn0IPXZrx9EEUMhizHrKri4klbOUE21ZCuqZgyKgQQrAfHk0yseL/b0JrQ4NGwFsL/59uf+9P9EU6v0Sx6LxJ3FqHTCiQD0C6wugyGBi9nfaeTOVM2AHzaD5e0iPGdzyxv/bdy1bKiDcweZPTri7BNLsbo3Em5QHiOVyBS6iENzf4lod6Q9CU5kVHQlyXFGI5ciuf/xLGD6d3Xo5op1oJHhb9Rca6vxzHpq+mXD6L0tRRrCL+6FH4h7D4tzfwej2pzkB4iG//yDlZneVpR95bq/3JfhsdqVKQEdkJvzrHhjB9WN31FIt2jQQ3wLRkF8iTeRE2x0xd06IvfawJAsqYOA

In [18]:
# Extract structured content from tool messages
for message in result["messages"]:
    if isinstance(message, ToolMessage) and message.artifact:
        structured_content = message.artifact["structured_content"]
        print(structured_content)

{'storeName': 'OVEN & FRIED CHICKEN', 'date': '09-02-2024', 'total': 51.55}
{'message': 'Check your receiptsExtraction table in Amazon DynamoDB'}
{'message': 'Check your email with subject Result of Receipt Extraction - OVEN & FRIED CHICKEN'}


In [19]:
# Invoke MCP server with sample photo file
result = await invokemcp("photos/pastaandco.jpg")
print(result)

Connecting to Receipt Extraction MCP Server...

Available tools: ['receiptExtraction', 'writeOutput', 'sendEmail']

Processing receipt photo file: photos/pastaandco.jpg
{'messages': [HumanMessage(content='Please extract the receipt information from photos/pastaandco.jpg, write the output to DynamoDB, and send an email notification.', additional_kwargs={}, response_metadata={}, id='7f74cb91-0282-44ae-8383-c5ea345f558a'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'receiptExtraction', 'arguments': '{"inputphoto": "photos/pastaandco.jpg"}'}, '__gemini_function_call_thought_signatures__': {'b42c1b4b-862c-4d44-ba32-d0e3f52766d7': 'CuMDAXLI2nxJni3dKM2n9bj6wt5GUapdfaFFU9aDkrazu37uE3ELozt4VGQtIBbUq7y9G2Mt9pbb573TkjmrQxsctM4Njhgq8/NUyhmimwqQjY5kz1akC7UsCBPtWkIYxehiUZd6WFh1Pz+D+f5AqEgEqtAi1RGuoETL1OOTs2trTjEdp01AiRxRE3I5jtT+9Il7HdvbUF3CuZi2+Y4m3eJieF1rNe3Yq5/7JfR3+3uvyWHSiATJnC984LvplR4b+6C9QqAs2mWn8iQ9gQu7eAtvx3LgVg7hjL2hshwmLwXdBSNhQ/GMkcTQqN5w/YJ8LPDrUjcT3kBSv/bNZccsh1

In [20]:
# Extract structured content from tool messages
for message in result["messages"]:
    if isinstance(message, ToolMessage) and message.artifact:
        structured_content = message.artifact["structured_content"]
        print(structured_content)

{'storeName': 'PASTA & CO.', 'date': '08-12-2023', 'total': 29.8}
{'message': 'Check your receiptsExtraction table in Amazon DynamoDB'}
{'message': 'Check your email with subject Result of Receipt Extraction - PASTA & CO.'}
